## Apredizaje por transferencia (Transfer Learning)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot(history):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].plot(history.history['loss'], label='Training loss')
    ax[0].plot(history.history['val_loss'], label='Validation loss')
    ax[0].legend()
    ax[1].plot(history.history['accuracy'], label='Training accuracy')
    ax[1].plot(history.history['val_accuracy'], label='Validation accuracy')

    ax[1].legend()

    ax[0].set_xlabel('Epochs')
    ax[1].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[1].set_ylabel('Accuracy')

    # Para visualizar mejor el loss respecto al cero
    ax[0].set_ylim(0, 1)


#### Uso del modelo VGG16 pre-entrenado.

In [ ]:
from keras.applications import VGG16

vgg16 = VGG16(weights='imagenet', # Coge los pesos preentrenados.
              include_top=False, # coge solo las capas convolucionales
              input_shape=(150, 150, 3)) # nuestra imagen es de 150x150

vgg16.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# congela las capas convolucionales
vgg16.trainable = False

model.summary()


NameError: name 'vgg16' is not defined

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def clasification(bath_size = 20, epochs = 20, img_weight = 150, img_height = 150, num_train_samples = 2000, num_validation_samples = 800):

    train_dir = 'Datasets/DogsCats/train'
    validation_dir = 'Datasets/DogsCats/validation'

    # compilamos en modelo definido anteriormente
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    # todos estos parametros son para enriquecer el dataset
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range=25,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.3,
                                       horizontal_flip=True,
                                       fill_mode='nearest')


    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_weight, img_height),
        batch_size= bath_size,
        class_mode='binary')

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_weight, img_height),
        batch_size= bath_size,
        class_mode='binary')


    history = model.fit(
        train_generator,
        steps_per_epoch= 4000 // bath_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps= 1200 // bath_size,
        verbose=2)

    return history

In [ ]:
history = clasification(num_train_samples=4000)

model.save("with_vgg16_cats_dogs.h5")
plot(history)

# NOS VAMOS A SARTAR EL FINE-TUNING DEL MODELO, LO UNICO QUE HACE ES DESOCNGELAR LAS CAPAS CONVOLUCIONALES ULTIMAS (ULTIMO BLOCK) Y DISMINUIR EL LEARNING RATE DEL RMSProp PARA ACABAR DE AFINAR EL MODELO

NameError: name 'clasification' is not defined